In [1]:
from utils.build_features import build_features
from utils.utils import get_klines, y_to_bins
from binance.client import Client
import pandas as pd
import numpy as np

from models.random_forest import RandomForest

C:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Hyper parameters:
CUTOFF = 0.15  # in percents. The minimal value of ascending
N_ESTIMATORS = 100


In [3]:
s_date = '01 Jan, 2019'
e_date = '03 Jan, 2019'
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_30MINUTE


In [5]:

kl_f_name = symbol + '_' + s_date + '_TO_' + e_date + '.csv'
# klines = get_klines(s_date, e_date, symbol, interval)
# klines.to_csv('klines/30M/' + kl_f_name, index=False)
klines = pd.read_csv('klines/30M/' + kl_f_name)

In [6]:
print(klines.shape)
klines.head()

(49, 7)


,timestamp,open,high,low,close,volume,Number of trades
0,1.546301e+12,3701.23,3713.00,3689.88,3700.31,686.367420,5534.0
1,1.546304e+12,3700.20,3702.73,3684.22,3689.69,613.539115,5086.0
2,1.546308e+12,3689.67,3695.95,3675.04,3690.00,895.302181,6391.0
3,1.546312e+12,3690.00,3699.77,3685.78,3693.13,796.714818,5709.0
4,1.546315e+12,3692.32,3720.00,3685.94,3692.71,1317.452909,7908.0


In [16]:


features_f_name = symbol + '_' + s_date + '_TO_' + e_date + '.csv'
# features = build_features(klines)
# features.to_csv('features/30M/' + features_f_name, index=False)
features = pd.read_csv('features/30M/' + features_f_name)



In [17]:
features['TO_PREDICT'] = features['close_ratio'].shift(-1)
features = y_to_bins(features, 'TO_PREDICT', CUTOFF)

In [18]:
features[['close', 'y_bins']].head(10)

,close,y_bins
0,3693.54,-1
1,3686.32,-1
2,3659.41,5
3,3696.73,1
4,3719.00,-1
5,3708.21,-1
6,3711.20,10
7,3797.14,-1
8,3778.02,-1
9,3770.00,1


In [29]:
l = features.shape[0]
X_train = np.array(features.iloc[:int(0.8 * l)].drop(['timestamp', 'y_bins'], axis=1))
X_valid = np.array(features.iloc[int(0.8 * l):].drop(['timestamp', 'y_bins'], axis=1))
y_train = np.array(features['y_bins'].iloc[:int(0.8 * l)])
y_valid = np.array(features['y_bins'].iloc[int(0.8 * l):])
df_valid = features.iloc[int(0.8 * l):]


In [30]:
model = RandomForest(n_estimators=N_ESTIMATORS)

In [31]:
model.fit(
    X=X_train,
    y=y_train)

In [32]:
df_valid['predictions'] = model.predict(df_valid.drop(['timestamp', 'y_bins'], axis=1))
pred_file_name = 'try.csv'
df_valid.to_csv('predictions/'+pred_file_name, index=False)

[0.44 0.35 0.13 0.08]
[0.67 0.22 0.05 0.06]
[0.86 0.09 0.05 0.  ]
[0.84 0.1  0.04 0.02]
[0.8  0.13 0.05 0.02]
[0.74 0.2  0.03 0.03]
[0.14 0.79 0.05 0.02]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
